In [1]:
import cmipld, glob, json, os, re
from rdflib import Graph as RGraph
from rdflib import Namespace, URIRef
from rdflib.namespace import NamespaceManager



prefix = cmipld.prefix()
cmipld.map_current(prefix)

prefix

Initializing LDR client...
Starting LDR server on port 3333...
Server started (PID: 90864)
LDR client initialized.
Initializing LDR client...
LDR client initialized.
Added mapping: https://emd.mipcvs.dev/ -> /Users/daniel.ellis/WIPwork/Essential-Model-Documentation/


'emd'

In [2]:
CONTEXTS = glob.glob('*/_context')

In [3]:
for cloc in CONTEXTS:
    folder = cloc.replace('_context','').rstrip('/')
    ctx = json.load(open(cloc))
    vocab = ctx['@context']["@vocab"]
    base = ctx['@context']["@base"]
    
    ids = sorted(os.popen(f'jq -r \'.["@id"]\' {folder}/*.json | grep -v null').read().splitlines())
    print(ids)
    
    ## JSONLD #################################
    graph = {**ctx, "@type": ["Collection"],"graph": [{"@id":i} for i in ids]}
    
    with open(f'{folder}/_graph.jsonld','w') as f:
        f.write(re.sub(r'\{\s+"@id":\s+"([^"]+)"\s+\}', r'{"@id": "\1"}',json.dumps(graph,indent=2)))
      
    ## RDF #################################
    data = cmipld.expand(graph,depth=2)
    rdfG = RGraph()
    # Reset namespace manager
    rdfG.namespace_manager = NamespaceManager(rdfG)

    # Bind prefixes to the graph
    rdfG.bind(None, Namespace(vocab),replace=True)
    rdfG.bind(f"{prefix}:{folder}", Namespace(base),replace=True)
    rdfG.bind("universe", Namespace("https://universal.mipcvs.dev/"),replace=True)
    # rdfG.bind("emd", emd_vocab)

    rdfG.parse(data=json.dumps(data[vocab+'graph']),
                base=vocab,
                format="json-ld")

    # Save to a local file (Turtle format is common)
    rdfG.serialize(destination=f"{folder}/_graph.ttl", format="turtle")
    # g2.parse("graph.ttl", format="turtle")     
    # pause    # 
    
    
    
    break
    

['access', 'arpege-climat', 'bcc-csm', 'bisicles', 'cam', 'canesm', 'cesm', 'clm', 'cnrm-cm', 'ec-earth', 'fgoals', 'gelato', 'geos', 'gfdl-cm4', 'giss-e2', 'hadam', 'hadcm2', 'hadgem3', 'icon', 'ifs', 'ipsl-cm', 'miroc', 'mpi-esm', 'nemo', 'nicam', 'pisces', 'reprobus', 'surfex', 'tactic', 'ukesm1']
[Cache MISS] {'@context': {'@alt_base': 'https://wcrp-cmip.github.io/Essential-Model-Documentation/model_family/', '@base': 'https://emd.mipcvs.dev/model_family/', '@esgvoc': 'ModelFamily', '@vocab': 'https://emd.mipcvs.dev/docs/contents/ModelFamily/', 'collaborative_institutions': {'@context': 'https://universal.mipcvs.dev/organisation/_context', '@type': '@id'}, 'primary_institution': {'@context': 'https://universal.mipcvs.dev/organisation/_context', '@type': '@id'}, 'references': {'@context': 'https://emd.mipcvs.dev/reference/_context', '@type': '@id'}, 'scientific_domains': {'@context': 'https://universal.mipcvs.dev/scientific_domain/_context', '@type': '@id'}}, '@type': ['Collection']

In [5]:
# Function to shorten URIs
def shorten(uri):
    if isinstance(uri, URIRef):
        try:
            return uri.n3(rdfG.namespace_manager)
        # rdfG.namespace_manager.qname(uri)
        except:
            return str(uri)
    return str(uri)

# Now print with shortened names
for s, p, o in rdfG:
    print(f"{shorten(s)} -- {shorten(p)} --> {shorten(o)}")

emd:model_family:canesm -- :description --> Canadian Earth System Model - developed by the Canadian Centre for Climate Modelling and Analysis.
emd:model_family:cesm -- :scientific_domains --> <https://universal.mipcvs.dev/scientific_domain/ocean>
emd:model_family:ec-earth -- :established --> none
emd:model_family:gfdl-cm4 -- :scientific_domains --> <https://universal.mipcvs.dev/scientific_domain/land_surface>
emd:model_family:hadgem3 -- :common_scientific_basis --> Non-hydrostatic atmospheric dynamics, primitive equation ocean dynamics, comprehensive land surface biogeophysics.
emd:model_family:geos -- :programming_languages --> none
emd:model_family:icon -- rdf:type --> <wcrp:model_family>
emd:model_family:canesm -- :license --> none
emd:model_family:gfdl-cm4 -- :documentation --> none
emd:model_family:cesm -- :common_scientific_basis --> none
emd:model_family:clm -- :programming_languages --> Python
emd:model_family:clm -- :scientific_domains --> <https://universal.mipcvs.dev/scienti

In [ ]:
simplufy further nto skip file name and clean up link . 
make in to a gexf. Save this. 
do we need a gexf script 


SyntaxError: invalid syntax (3322917219.py, line 1)

In [6]:
self = ctx['@context']['@esgvoc']


import requests,json
def getc(x):
    if 'http' in x:
        
        x = '/'.join(x.split('/')[:-1])+'/_context'
        print(x)
        
        dt = requests.get(x).json().get("@esgvoc",x)
        print(dt)
        return dt
    
# for s, p, o in rdfG:
#     p = shorten(p)
    
#     if p[0] == ':':
#         s = getc(s)
#         o = getc(o)
#         if o: 
#             print(f"{shorten(s)} -- {p} --> {o}")
    

print(cmipld.client.test_load("https://emd.mipcvs.dev/model_family/_context")['preview'])

{"@context":{"@alt_base":"https://wcrp-cmip.github.io/Essential-Model-Documentation/model_family/","@base":"https://emd.mipcvs.dev/model_family/","@esgvoc":"ModelFamily","@vocab":"https://emd.mipcvs.dev/docs/contents/ModelFamily/","collaborative_institutions":{"@context":"https://universal.mipcvs.dev/organisation/_context","@type":"@id"},"primary_institution":{"@context":"https://universal.mipcvs.dev/organisation/_context","@type":"@id"},"references":{"@context":"https://emd.mipcvs.dev/reference...


In [7]:
# Print all namespaces in the graph
for prefix, namespace in rdfG.namespaces():
    print(f"{prefix}: {namespace}")

brick: https://brickschema.org/schema/Brick#
csvw: http://www.w3.org/ns/csvw#
dc: http://purl.org/dc/elements/1.1/
dcat: http://www.w3.org/ns/dcat#
dcmitype: http://purl.org/dc/dcmitype/
dcterms: http://purl.org/dc/terms/
dcam: http://purl.org/dc/dcam/
doap: http://usefulinc.com/ns/doap#
foaf: http://xmlns.com/foaf/0.1/
geo: http://www.opengis.net/ont/geosparql#
odrl: http://www.w3.org/ns/odrl/2/
org: http://www.w3.org/ns/org#
prof: http://www.w3.org/ns/dx/prof/
prov: http://www.w3.org/ns/prov#
qb: http://purl.org/linked-data/cube#
schema: https://schema.org/
sh: http://www.w3.org/ns/shacl#
skos: http://www.w3.org/2004/02/skos/core#
sosa: http://www.w3.org/ns/sosa/
ssn: http://www.w3.org/ns/ssn/
time: http://www.w3.org/2006/time#
vann: http://purl.org/vocab/vann/
void: http://rdfs.org/ns/void#
wgs: https://www.w3.org/2003/01/geo/wgs84_pos#
owl: http://www.w3.org/2002/07/owl#
rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs: http://www.w3.org/2000/01/rdf-schema#
xsd: http://www.w3.

In [34]:
# Function to shorten URIs
def shorten(uri):
    if isinstance(uri, URIRef):
        uri = '/'.join(uri.split('/')[:-1])
        try:
            if 'mipcvs.dev' not in uri:
                # print(uri)
                return uri.n3(rdfG.namespace_manager)
            return rdfG.namespace_manager.qname(uri)
        except:
            return str(uri)
    return str(uri)

# Now print with shortened names

links = []
for s, p, o in rdfG:
    if 'mipcvs.dev' in s and 'mipcvs.dev' in o:
        # links.append([shorten(s),shorten(p).replace(':',''),shorten(o)])
        links.append(f"{shorten(s)} -- {shorten(p).replace(':','')} --> {shorten(o)}")

In [35]:
# import counter
from collections import Counter
Counter(links)

Counter({'ns1:model_family -- ns2ModelFamily --> universe:scientific_domain': 75,
         'ns1:model_family -- ns2ModelFamily --> universe:organisation': 44,
         'ns1:model_family -- http//www.w3.org/1999/02 --> ns2:ModelFamily': 30,
         'ns1:model_family -- ns2ModelFamily --> ns1:reference': 4})